In [ ]:
import geopandas as gpd
import pandas as pd
from ribasim_nl import CloudStorage
from ribasim_nl.geodataframe import join_by_poly_overlay, split_basins
from ribasim_nl.raster import sample_level_area

cloud = CloudStorage()

## Bewerken krw oppervlaktewaterlichamen tot basins
- Opsplitsen van `krw_oppervlaktewaterlichamen_nederland_vlakken.gpkg` met `krw_split_lijnen.gpkg`
- Toevoegen van `waterlichaam` namen uit `oppervlaktewaterlichamen_rijk.gpkg`

In [ ]:
exclude_owmident = [
    "NL95_1A",
    "NL95_2A",
    "NL95_3A",
    "NL95_4A",
    "NL81_1",
    "NL81_10",
    "NL81_3",
    "NL81_2",
    "NL89_zwin",
]
krw_poly_gdf = gpd.read_file(
    cloud.joinpath(
        "Basisgegevens", "KRW", "krw_oppervlaktewaterlichamen_nederland_vlakken.gpkg"
    )
)

krw_poly_gdf = krw_poly_gdf.loc[~krw_poly_gdf.owmident.isin(exclude_owmident)]
krw_poly_gdf = krw_poly_gdf.explode(index_parts=False)
krw_poly_gdf["shapeArea"] = krw_poly_gdf.area
krw_poly_gdf.sort_values("shapeArea", ascending=False)
krw_poly_gdf.drop_duplicates("owmident", keep="first", inplace=True)


krw_split_lines_gdf = gpd.read_file(
    cloud.joinpath("Rijkswaterstaat", "verwerkt", "model_user_data.gpkg"),
    layer="krw_split_lijnen",
)


rws_opp_poly_gdf = gpd.read_file(
    cloud.joinpath(
        "Rijkswaterstaat", "aangeleverd", "oppervlaktewaterlichamen_rijk.gpkg"
    )
)


rws_krw_poly_gdf = split_basins(krw_poly_gdf, krw_split_lines_gdf)


rws_krw_poly_gdf = join_by_poly_overlay(
    rws_krw_poly_gdf,
    rws_opp_poly_gdf[["waterlichaam", "geometry"]],
    select_by="poly_area",
)


rws_krw_poly = cloud.joinpath("Rijkswaterstaat", "verwerkt", "krw_basins_vlakken.gpkg")

rws_krw_poly_gdf["basin_id"] = rws_krw_poly_gdf.reset_index().index + 1
rws_krw_poly_gdf.to_file(rws_krw_poly)

## Aanmaken hoogte-oppervlakte-relaties

In [ ]:
raster_path = cloud.joinpath(
    "Rijkswaterstaat", "aangeleverd", "bathymetrie", "bathymetrie.vrt"
)

dfs = [
    sample_level_area(raster_path, row.geometry, ident=row.basin_id)
    for row in rws_krw_poly_gdf.itertuples()
]

df = pd.concat(dfs)

df.to_csv(
    cloud.joinpath("Rijkswaterstaat", "verwerkt", "krw_basins_vlakken_level_area.csv")
)